## Objetivo

Após a escolha da implementação dos modelos sobre dados potencialmente diferentes do treino, i.e., a escolha por deixar de aplicar as predições sobre reviews de produtos da Amazon para então aplicar a reviews de jogos da Steam, consideramos a necessidade de estudo sobre a capacidade de generalização de nossos modelos para o tipo de review que aparecerá em produção, de modo a evitar possíveis problemas graves de Data Drift.

Em caso de insatisfatoriedade da performance dos modelos nos dados da Steam, teremos de retreiná-los sobre as reviews da Steam.

Dito isso, neste notebook, vamos realizar as seguintes tarefas: 

1) Obter e preparar dados de reviews da Steam para a validação (separando uma base de treino a ser utilizada em caso de necessidadade).

2) A partir dos diversos modelos pré-treinados, embedar as reviews para validação em diferentes datasets de teste.

3) Testar os 105 classificadores até então treinados nos datasets de teste obtidos.

4) Avaliar o desempenho desses modelos no novo domínio, comparando suas métricas, especialmente AUC, entre os dados da Steam e da Amazon, com foco em identificar possíveis quedas significativas de performance.

5) Caso a performance dos modelos pré-treinados se revele insatisfatória, retreinar os classificadores diretamente sobre embeddings das reviews da Steam, mantendo o mesmo rigor metodológico adotado anteriormente.

6) Comparar os modelos treinados na Steam com os anteriores, buscando identificar qual abordagem entrega melhor generalização e custo-benefício para aplicação em produção.

O objetivo final é selecionar o modelo mais adequado para lidar com reviews da Steam, assegurando estabilidade e eficácia do sistema preditivo em um cenário distinto do conjunto de treinamento original.

## Imports

In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import sys
import os
import mlflow
import re
from sklearn.model_selection import train_test_split
sys.path.append(os.path.abspath(".."))
from src import reduce_text_df_per_class, PyCaretEmbeddingClassificationTrainer, BinaryClassificationEvaluator

device = "cuda" if torch.cuda.is_available() else "cpu"

## Obter e preparar dados

In [4]:
raw_data_path = '../data/steam_data/raw'
processed_data_path = '../data/steam_data/processed'
os.makedirs(raw_data_path, exist_ok=True)
os.makedirs(processed_data_path, exist_ok=True)

kaggle_api = KaggleApi()
kaggle_api.authenticate()
kaggle_api.dataset_download_files('andrewmvd/steam-reviews', path=raw_data_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/steam-reviews


In [5]:
df = pd.read_csv(os.path.join(raw_data_path, 'dataset.csv'))
print(df.shape)
df.info()

(6417106, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6417106 entries, 0 to 6417105
Data columns (total 5 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   app_id        int64 
 1   app_name      object
 2   review_text   object
 3   review_score  int64 
 4   review_votes  int64 
dtypes: int64(3), object(2)
memory usage: 244.8+ MB


In [6]:
df.head()

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [7]:
#Verificando nulos
print(df.isnull().sum())

app_id               0
app_name        183234
review_text       7305
review_score         0
review_votes         0
dtype: int64


In [8]:
df.dropna(inplace=True)

In [9]:
print(df['review_score'].value_counts())
print(df['review_votes'].value_counts())

review_score
 1    5126132
-1    1100596
Name: count, dtype: int64
review_votes
0    5313079
1     913649
Name: count, dtype: int64


De acordo com a página do kaggle:

app_id = Game ID,

app_name = Game Name,

review_text = Review text,

review_score = Review Sentiment: whether the game the review recommends the game or not,

review_votes = Review vote: whether the review was recommended by another user or not.

Bastanto atentar ao head, parece estar incorreto inferir diretamente sentimento positivo da recomendação de um player. Uma forma de reduzir a amostra e ao mesmo tempo filtrar reviews potencialmente mais consistentes em suas recomendações é excluir todas as reviews que não receberam votos. Faremos isso, além de adaptar a categoria review_score ao padrão de 0 e 1 para sentimentos positivo e negativo que viemos usando. Na sequência, usaremos estratégias de balanceamento e avaliaremos se o tamanho da amostra está conveniente.



In [10]:
df = df[df['review_votes'] > 0]
df = df.drop(columns=['review_votes'])
df['review_score'] = df['review_score'].replace(-1, 0)

print(df.shape)
print(df['review_score'].value_counts())

(913649, 4)
review_score
1    649288
0    264361
Name: count, dtype: int64


Vamos remover duplicatas em review_text.

In [11]:
df = df.drop_duplicates(subset=['review_text'])
print(df.shape)
print(df['review_score'].value_counts())

(661317, 4)
review_score
1    465875
0    195442
Name: count, dtype: int64


Uma técnica conveniente para que os nomes de games muito detestados ou muito amados pela comunidade não influenciem na análise de sentimento simplesmente por serem mencionados na review é substituir strings em review_text parecidas com o que consta na coluna app_name pela máscara genérica "this game". Vamos implementar essa técnica a seguir. 

In [ ]:
# Função para substituir o nome do jogo por "this game"
def mask_game_name(review_text, app_name):
    if pd.isna(review_text) or pd.isna(app_name):
        return review_text
    # Escapar caracteres especiais no app_name para criar um padrão seguro de regex
    escaped_app_name = re.escape(app_name)
    # Substituir todas as ocorrências do app_name ignorando maiúsculas/minúsculas
    pattern = re.compile(rf'\b{escaped_app_name}\b', flags=re.IGNORECASE)
    return pattern.sub('this game', review_text)

print(f"Antes do masking:{df.iloc[10000]}")
df['review_text'] = df.apply(lambda row: mask_game_name(row['review_text'], row['app_name']), axis=1)
print(f"Depois do masking:{df.iloc[10000]}")

#dropando as colunas que não serão mais utilizadas
df = df.drop(columns=['app_id','app_name'])

Antes do masking:app_id                                                     105600
app_name                                                 Terraria
review_text     terraria is a good game even tho countless peo...
review_score                                                    1
Name: 77057, dtype: object
Depois do masking:app_id                                                     105600
app_name                                                 Terraria
review_text     this game is a good game even tho countless pe...
review_score                                                    1
Name: 77057, dtype: object


## Underbalancing e Train/Test Split

Utilizaremos os embeddings das reviews para reamostragem com o MinibatchKMeans, como feito anteriormente. O objetivo é manter a comparabilidade com os treinamentos realizados no sample de 0,5% dos dados da Amazon. Para isso, trabalharemos com 20.000 linhas simetricamente balanceadas entre treino e teste. O balanceamento será feito ajustando diretamente o número de clusters por classe, totalizando 40.000 linhas, a serem divididas entre treino e teste.


In [16]:
df_reduced = reduce_text_df_per_class(
    df=df,
    class_column="review_score",
    embedding_input='review_text', #podemos informar a coluna com o texto no argumento para vetorização a partir da própria função
    embedding_model_path="../models/sbert_models/all-MiniLM-L6-v2",
    device=device,
    target_clusters=20000
)


2025-05-05 14:20:59,964 - INFO - Generating embeddings...


Batches:   0%|          | 0/2584 [00:00<?, ?it/s]

2025-05-05 14:27:35,519 - INFO - Processing class: 1
2025-05-05 14:27:35,745 - INFO - Class size: 465875 → Clusters: 20000
2025-05-05 14:27:35,746 - INFO - Fitting MiniBatchKMeans...
2025-05-05 14:47:27,367 - INFO - Processing class: 0
2025-05-05 14:47:27,556 - INFO - Class size: 195442 → Clusters: 20000
2025-05-05 14:47:27,557 - INFO - Fitting MiniBatchKMeans...
2025-05-05 15:07:26,380 - INFO - Final reduced dataset size: (40000, 2)
2025-05-05 15:07:26,386 - INFO - 
review_score
0    20000
1    20000
Name: count, dtype: int64


Executaremos a separação de teste e treino:

In [19]:

df_train, df_test = train_test_split(
    df_reduced,
    test_size=0.5,
    stratify=df_reduced['review_score'],
    random_state=42
)

# Exibir as proporções
print("Distribuição proporcional de 'review_score':")

print("\nTreino:")
print(df_train['review_score'].value_counts())

print("\nTeste:")
print(df_test['review_score'].value_counts())

df_train.to_csv("../data/steam_data/processed/train_steamreviews_sample.csv", index=False)
df_test.to_csv("../data/steam_data/processed/test_steamreviews_sample.csv", index=False)


Distribuição proporcional de 'review_score':

Treino:
review_score
1    10000
0    10000
Name: count, dtype: int64

Teste:
review_score
0    10000
1    10000
Name: count, dtype: int64


## Vetorizando os dados com Modelos Pré-Treinados

In [20]:
models_dir = "../models/sbert_models/"
data_dir = "../data/steam_data/processed/"
output_dir = os.path.join(data_dir, "embedded_data")
os.makedirs(output_dir, exist_ok=True)

TEXT_COL = "review_text"
CLASS_COL = "review_score"

model_names = [m for m in os.listdir(models_dir) if os.path.isdir(os.path.join(models_dir, m))]
dataset_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]

for model_name in tqdm(model_names, desc="Modelos"):
    model = SentenceTransformer(os.path.join(models_dir, model_name), device="cuda" if torch.cuda.is_available() else "cpu")

    for dataset_file in dataset_files:
        df = pd.read_csv(os.path.join(data_dir, dataset_file))

        texts = df[TEXT_COL].astype(str).tolist()
        embeddings = model.encode(texts, batch_size=256, show_progress_bar=False, normalize_embeddings=True)

        emb_df = pd.DataFrame(embeddings, columns=[f"CLS{i}" for i in range(len(embeddings[0]))])
        emb_df[TEXT_COL] = df[TEXT_COL]
        emb_df[CLASS_COL] = df[CLASS_COL]

        output_name = f"{os.path.splitext(dataset_file)[0]}_{model_name}.csv"
        emb_df.to_csv(os.path.join(output_dir, output_name), index=False)

Modelos:   0%|          | 0/7 [00:00<?, ?it/s]

## Avaliação dos Classificadores do treinamento anterior sobre os dados da Steam

Avaliaremos, antes de seguir com o treinamento, como os classificadores treinados sobre os dados da Amazon desempenham sobre o sentimento pertinente às reviews dos jogadores da Steam.

Primeiro, conectamos ao servidor MLflow e recuperamos as execuções do experimento "pycaret-embeddings-classification"

In [7]:
client = mlflow.tracking.MlflowClient()
experiment_id = mlflow.get_experiment_by_name("pycaret-embeddings-classification").experiment_id
runs = client.search_runs(experiment_ids=experiment_id)

Aqui, preparamos os arquivos de dados e configuramos as variáveis necessárias para o processamento. Especificamos os diretórios e arquivos de dados, incluindo os arquivos de treino e teste, e configuramos quais colunas serão utilizadas no modelo, como a coluna alvo (review_score) e as colunas a serem descartadas (review_text).

Esta célula será aproveitada também na iteração de treino sobre os dados da Steam.

In [2]:
embedded_data_dir = "../data/steam_data/processed/embedded_data"
target_column  = "review_score"
drop_columns = ["review_text"]

all_files = [f for f in os.listdir(embedded_data_dir) if f.endswith('.csv')]
train_files = [f for f in all_files if f.startswith('train')]
test_files = [f for f in all_files if f.startswith('test')]

def get_embedder_suffix(filename):
    return filename.split('_', 3)[-1]

test_lookup = {get_embedder_suffix(f): f for f in test_files}

Nesta parte, buscamos os arquivos de teste e associamos cada um ao modelo treinado que foi utilizado para criar o embedding. Depois, usamos esses modelos para avaliar seu desempenho nos dados de teste, registrando os resultados de forma organizada.


In [4]:
for suffix, test_file in tqdm(test_lookup.items(), desc='Datasets de Teste'):
    
    embedding_model_name = suffix.replace('.csv', '')
    df_test = pd.read_csv(os.path.join(embedded_data_dir, test_file))
    
    # Filtrar apenas runs com o embedding model correspondente ao do dataset de teste
    matching_runs = [
        run for run in runs 
        if run.data.tags.get('embedding_model_name') == embedding_model_name
    ]
    
    for run in matching_runs:
        run_id = run.info.run_id

        with mlflow.start_run(run_id=run_id):
            model_uri = f"runs:/{run_id}/model"
            model = mlflow.sklearn.load_model(model_uri)

            evaluator = BinaryClassificationEvaluator(
                model=model,
                test_dataset_name=test_file.replace('.csv', ''),
                test_dataset_prefix="steamreviews"
            )

            metrics = evaluator.evaluate_sklearn_model(
                df_test,
                target_column=target_column,
                drop_columns=drop_columns
            )

            evaluator.log_to_mlflow(metrics)


Datasets de Teste:   0%|          | 0/7 [00:00<?, ?it/s]

Vamos agora extrair os dados sobre os modelos e suas métricas de desempenho, organizando essas informações em uma tabela para avaliar se houve quedas de desempenho de um tipo de review a outro.

In [ ]:
def extract_mlflow_metrics(runs, prefix_test_metric='amazonreviews'):

    mlflow_data = []

    for run in runs:
        info = {}
        info['run_id'] = run.info.run_id

        # Tags
        tags = run.data.tags
        train_name = tags.get('train_dataset_name', '')
        parts = train_name.split('_')
        info['train_dataset'] = '_'.join(parts[2:3]) if len(parts) >= 3 else ''
        info['embedding_model'] = tags.get('embedding_model_name')
        info['model_name'] = tags.get('model_name')

        # Métricas
        for metric_name, value in run.data.metrics.items():
            if metric_name.startswith('test'):
                parts = metric_name.split('_')
                new_metric_name = prefix_test_metric + '_'.join(parts[-1:])
                info[new_metric_name] = value
            else:
                info[metric_name] = value

        mlflow_data.append(info)

    return pd.DataFrame(mlflow_data)

mlflow_df = extract_mlflow_metrics(runs)

print(mlflow_df.shape)

columns = ['run_id', 'train_dataset', 'embedding_model', 'model_name', 'amazonreviewsTestAUC', 'steamreviewsTestAUC']
filtered_mlflow_data = mlflow_df[columns].sort_values(by='steamreviewsTestAUC', ascending=False)
filtered_mlflow_data.reset_index(inplace=True, drop=True)
filtered_mlflow_data.index = filtered_mlflow_data.index + 1
filtered_mlflow_data.to_excel("../reports/AmazonTrain_SteamTest.xlsx")
filtered_mlflow_data.head(30)

(105, 24)


,run_id,train_dataset,embedding_model,model_name,amazonreviewsTestAUC,steamreviewsTestAUC
1,87d75ef72bad4c19946559891b71a013,sample0005,gte-small,LogisticRegression,0.993981,0.925512
2,f7b622218ef1454e94b8aa6f41c52807,sample0010,gte-small,GaussianNB,0.993225,0.925449
3,db866db06979460b84ca82b283ff9bf5,sample0020,gte-small,GaussianNB,0.993179,0.925390
4,04640f874deb4c73bca58a1a3ff35b09,sample0020,bge-base-en-v1.5,ExtraTreesClassifier,0.992970,0.925352
5,b3ebca80e8d34cbcbe6f383cdc61246a,sample0005,gte-small,GaussianNB,0.993097,0.925264
6,9212f59735a3429584ba058798a3f8ad,sample0010,gte-small,LogisticRegression,0.994086,0.924350
7,c604ce41a58f4cfa8402e615984f2e16,sample0020,gte-small,LogisticRegression,0.994121,0.924344
8,f54c5bc5c20b4badaccb667433c0b02c,sample0010,bge-base-en-v1.5,ExtraTreesClassifier,0.992921,0.923971
9,fdbb7d2336d344a582e4d304555711b0,sample0005,bge-base-en-v1.5,LogisticRegression,0.993728,0.923863
10,c3c694ffacfb40bf8d15d3bd898fbadf,sample0005,bge-base-en-v1.5,ExtraTreesClassifier,0.992817,0.923507


Para nossa feliz surpresa a Regressão Logística campeã da última análise segue liderando entre os demais classificadores na AUC de teste também sobre os dados da Steam, e apesar de ter havido uma queda de performance, ela não foi muito alta, com a queda em apenas 0.07 pontos de AUC, e o mesmo pode-se dizer sobre a queda de performance dos demais classificadores, ao menos entre os top 15, que demonstraram capacidade maior de generalização evidentemente em virtude dos embedders subjacentes, bge-base-en-v1.5 e o gte-small.

Poderíamos encerrar aqui a análise, mas temos interesse em investigar como nosso processo de treinamento se aplicado aos dados da Steam pode obter resultados distintos tanto no dataset de teste da Steam como no da Amazon.

## Treino e avaliação dos classificadores sobre os dados da Steam

Inicialmente vamos configurar o acesso aos dados vetorizados de teste da amazon para, na sequência, colocarmos em prática nossa pipeline de treino dos classificadores sobre os embeddings da Steam com teste em ambos os dados da Steam e da Amazon.

In [3]:
amazon_embedded_data_dir = "../data/processed/embedded_data"
amazon_target_column = 'class'
amazon_drop_columns = ['text']

amazon_all_files = [f for f in os.listdir(amazon_embedded_data_dir) if f.endswith('.csv')]
amazon_test_files = {f.replace('train', 'test'): os.path.join(amazon_embedded_data_dir, f) for f in amazon_all_files if f.startswith('test')}

amazon_test_lookup = {get_embedder_suffix(f): f for f in test_files}

#Linha necessária devido à mudança do padrão no caso dos arquivos de teste da Amazon
amazon_test_lookup = {k: v.replace("steamreviews_sample", "amazonreviews_sample0050") for k, v in amazon_test_lookup.items()}

In [4]:
for train_file in tqdm(train_files, desc='Datasets de Embeddings'):
    
    suffix = get_embedder_suffix(train_file)
    test_file = test_lookup.get(suffix)
    amazon_test_file = amazon_test_lookup.get(suffix)

    df_train = pd.read_csv(os.path.join(embedded_data_dir, train_file))
    df_test = pd.read_csv(os.path.join(embedded_data_dir, test_file))
    df_amazon = pd.read_csv(os.path.join(amazon_embedded_data_dir, amazon_test_file))

    trainer = PyCaretEmbeddingClassificationTrainer(
        train_dataset=df_train,
        target_column=target_column,
        drop_columns=drop_columns,  
    )

    trained_models = trainer.train()

    run_ids = trainer.log_to_mlflow(
        add_tags={"train_dataset_name": train_file.replace('.csv', ''),
                "embedding_model_name": suffix.replace('.csv', ''),},
    )
    
    for run_id in run_ids:

        with mlflow.start_run(run_id=run_id):
            model_uri = f"runs:/{run_id}/model"
            model = mlflow.sklearn.load_model(model_uri)

            evaluator = BinaryClassificationEvaluator(
                model=model,
                test_dataset_name=test_file.replace('.csv', ''),
                test_dataset_prefix="steamreviews"
            )

            metrics = evaluator.evaluate_sklearn_model(
                df_test,
                target_column=target_column,
                drop_columns=drop_columns
            )

            evaluator.log_to_mlflow(metrics)

            # Avaliação na Amazon
            amazon_evaluator = BinaryClassificationEvaluator(
                model=model,
                test_dataset_name=amazon_test_file.replace('.csv', ''),
                test_dataset_prefix="amazonreviews"
            )

            amazon_metrics = amazon_evaluator.evaluate_sklearn_model(
                df_amazon,
                target_column=amazon_target_column,
                drop_columns=amazon_drop_columns
            )

            amazon_evaluator.log_to_mlflow(amazon_metrics)

Datasets de Embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

E aqui veremos as métricas resultantes, conforme acima:

In [10]:
runs = client.search_runs(experiment_ids=experiment_id)
mlflow_df = extract_mlflow_metrics(runs)
print(mlflow_df.shape)

columns = ['run_id', 'train_dataset', 'embedding_model', 'model_name', 'amazonreviewsTestAUC', 'steamreviewsTestAUC']
filtered_mlflow_data = mlflow_df[columns].sort_values(by='steamreviewsTestAUC', ascending=False)
filtered_mlflow_data.reset_index(inplace=True, drop=True)
filtered_mlflow_data.index = filtered_mlflow_data.index + 1
filtered_mlflow_data.to_excel("../reports/SteamTrain_Results.xlsx")
filtered_mlflow_data.head(30)


(140, 24)


,run_id,train_dataset,embedding_model,model_name,amazonreviewsTestAUC,steamreviewsTestAUC
1,d63eafd08d7f41008759ed0f81efd71c,sample,bge-base-en-v1.5,LogisticRegression,0.990425,0.941958
2,f2b712e5b1544fd189b1746a58b70d94,sample,bge-base-en-v1.5,LGBMClassifier,0.987676,0.941911
3,a6e9481bcbb246938bc58996c09c82ed,sample,bge-base-en-v1.5,LinearDiscriminantAnalysis,0.985695,0.939460
4,76bceb6b85914aa59da16f77472c86aa,sample,gte-small,LogisticRegression,0.991654,0.939322
5,d791038f7ac945d391af8d25e27114de,sample,gte-small,LGBMClassifier,0.986206,0.938898
6,f30900751330402fbbebb89342c70666,sample,gte-small,ExtraTreesClassifier,0.986664,0.937231
7,87d75ef72bad4c19946559891b71a013,sample0005,gte-small,LogisticRegression,0.993981,0.925512
8,f7b622218ef1454e94b8aa6f41c52807,sample0010,gte-small,GaussianNB,0.993225,0.925449
9,db866db06979460b84ca82b283ff9bf5,sample0020,gte-small,GaussianNB,0.993179,0.925390
10,04640f874deb4c73bca58a1a3ff35b09,sample0020,bge-base-en-v1.5,ExtraTreesClassifier,0.992970,0.925352


Finalmente, o resultado foi que apenas por uma pequena diferença a regressão logística treinada sobre os dados vetorizados da Steam pelo gte-small superou o treinamento sobre os dados da Amazon. Como a queda de performance sobre os dados da Amazon não foi significativa, o modelo denota preferibilidade em termos de generalização para análise de sentimento de diferentes públicos com relação ao anterior, ainda que por pouco.

Embora os primeiros lugares tenham cabido a três classificadores treinados em vetores da Steam obtido a partir do bge-base-en-v1.5, será no entanto à mencionada regressão logística que daremos o título de campeão do nosso experimento, uma vez que provou-se menos custoso em produção fazer uso de vetores do gte-small, e a diferença de performance entre ambos é mínima. O modelo que utilizaremos em produção já não sertá mais então o de run_id 87d75ef72bad4c19946559891b71a013, mas sim o quarto colocado da tabela acima, de run_id 76bceb6b85914aa59da16f77472c86aa.

## Teste de predições a partir do modelo selecionado

Faremos abaixo alguns testes de predição com o modelo selecionado, uma utilizando a biblioteca do mlflow para acessar o modelo, e outro fazendo uso do joblib, como alternativa sugerida pelo sklearn.

In [12]:
batch_size = 8

def get_df_embeddings(_embedding_model, text):
    embeddings = _embedding_model.encode(text,batch_size=batch_size)
    df_embeddings = pd.DataFrame(embeddings, columns=[f"CLS{i}" for i in range(embeddings.shape[1])])
    return df_embeddings

df = pd.read_csv("../data/steam_data/processed/test_steamreviews_sample.csv")
df = df.sample(1000, random_state=5).reset_index(drop=True)

import time
start_time = time.time()

df_embeddings = get_df_embeddings(
    _embedding_model=SentenceTransformer("../models/sbert_models/gte-small",device='cpu'),
    text=df["review_text"].astype(str).tolist()
)

end_time = time.time()
print(f"Tempo para cálculo dos embeddings com batch_size={batch_size}: {end_time - start_time} segundos.")

Tempo para cálculo dos embeddings com batch_size=8: 13.940841674804688 segundos.


In [ ]:
# Melhorando a exibição das reviews abaixo.
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [38]:
import mlflow.sklearn

model_uri = "runs:/76bceb6b85914aa59da16f77472c86aa/model"

model_sklearn_call = mlflow.sklearn.load_model(model_uri)

preds = model_sklearn_call.predict(df_embeddings)
probs = model_sklearn_call.predict_proba(df_embeddings)

predictions_sklearn = pd.DataFrame({
    'Label': preds,
    'Score_0': probs[:, 0],
    'Score_1': probs[:, 1]
})

predictions_sklearn = pd.concat([df[["review_text", "review_score"]], predictions_sklearn], axis=1)
predictions_sklearn.head(5)


,review_text,review_score,Label,Score_0,Score_1
0,"Love it, but hackers is a big problem",1,1,0.445784,0.554216
1,R* has lost my support with the way they are treating the summer sale. No one gives a ♥♥♥♥ about in-game money attached to the sale with a -25% off tag. Awful way to run things.,0,0,0.977318,0.022682
2,"Great Game! If you love classic 'Point and Click' Adventures,lovely designed Level and beautiful Music you shouldn't miss this game. The riddles may be confusing sometimes and need a certain amount of brain to solve but if you like to combine a snail with an envelope you will have no real problem but instead lots of fun.",1,1,0.026411,0.973589
3,One of the most frustrating games I've ever played... And I can't stop! Love it.,1,1,0.211949,0.788051
4,"Defense Grid is 'Babby's first Tower Defense' and possibly the ♥♥♥♥♥iest Tower Defense clone ever created. A pure pile of ♥♥♥♥. If you're looking for a Tower Defense game, play the original flash games or try the game Sanctum.",0,1,0.223798,0.776202


In [39]:
import joblib

model_path = r"..\mlruns\294574624479352871\76bceb6b85914aa59da16f77472c86aa\artifacts\model\model.pkl"
model_sklearn_call = joblib.load(model_path) 
preds = model_sklearn_call.predict(df_embeddings)
probs = model_sklearn_call.predict_proba(df_embeddings)

predictions_sklearn = pd.DataFrame({
    'Label': preds,
    'Score_0': probs[:, 0],
    'Score_1': probs[:, 1]
})

predictions_sklearn = pd.concat([df[["review_text", "review_score"]], predictions_sklearn], axis=1)
predictions_sklearn.head(5)


,review_text,review_score,Label,Score_0,Score_1
0,"Love it, but hackers is a big problem",1,1,0.445784,0.554216
1,R* has lost my support with the way they are treating the summer sale. No one gives a ♥♥♥♥ about in-game money attached to the sale with a -25% off tag. Awful way to run things.,0,0,0.977318,0.022682
2,"Great Game! If you love classic 'Point and Click' Adventures,lovely designed Level and beautiful Music you shouldn't miss this game. The riddles may be confusing sometimes and need a certain amount of brain to solve but if you like to combine a snail with an envelope you will have no real problem but instead lots of fun.",1,1,0.026411,0.973589
3,One of the most frustrating games I've ever played... And I can't stop! Love it.,1,1,0.211949,0.788051
4,"Defense Grid is 'Babby's first Tower Defense' and possibly the ♥♥♥♥♥iest Tower Defense clone ever created. A pure pile of ♥♥♥♥. If you're looking for a Tower Defense game, play the original flash games or try the game Sanctum.",0,1,0.223798,0.776202


## Conclusão

Encerramos esta análise confirmando o objetivo central do notebook: investigar a generalização de modelos de análise de sentimento treinados sobre reviews da Amazon quando aplicados a dados da Steam, bem como avaliar o desempenho de modelos treinados diretamente sobre os dados desta última. Demonstramos que, embora exista uma leve queda de desempenho na transferência entre domínios, especialmente na métrica AUC, classificadores como a Regressão Logística — notadamente em embeddings como os do gte-small — sustentaram desempenho competitivo e com baixa perda de eficácia, destacando-se tanto em robustez quanto em custo computacional. Por fim, elegemos como modelo de produção aquele com run_id 76bceb6b85914aa59da16f77472c86aa, não apenas por sua performance consistente, mas também por seu equilíbrio entre eficiência e capacidade de generalização, com as métricas finais: amazonreviewsTestAUC=0.991654 e steamreviewsTestAUC=0.939322.
